In [12]:
import codecs
from dataclasses import dataclass
from typing import List, Dict, Any

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch import Tensor
from torch import nn
import time
import pickle
import datetime
import random

# Ensuring reproducibility
seed = 0
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)
random.seed(seed)

import sys
sys.path.insert(1, '/work/nlp-project')
from scripts.read_write_data import read_processed_data, read_raw_data, load_data
from models.classes import DataIterator, Batch, PolyDataIterator, F1_evaluator, F1_error_evaluator, Train2BiLSTM

import gensim.models
GoogleEmbs = gensim.models.KeyedVectors.load_word2vec_format(
                                '/work/nlp-project/models/GoogleNews-50k.bin', binary=True)

In [3]:
PP_TRAIN_SET_PATH = "nlp-project/data/pseudo/pseudo.conll"
TRAIN_SET_PATH = "nlp-project/data/processed/train_splits/labeled.conll"
DEV_SET_PATH = "nlp-project/data/processed/dev.conll"

In [4]:
documents, doc_labels, _, _ = load_data(TRAIN_SET_PATH)
dev_docs, dev_labels, _, _ = load_data(DEV_SET_PATH)


pp_documents = []
pp_doc_labels = []
for words, labels in read_raw_data(PP_TRAIN_SET_PATH):
    pp_documents.append(words)
    pp_doc_labels.append(labels)

In [0]:
model = Train2BiLSTM(hidden_size=10)
model.fit(train=(documents, doc_labels),
          train2=(pp_documents, pp_doc_labels),
          print_metrics=False, 
          learning_rate=0.001,
          dev = (dev_docs, dev_labels),
          epochs=20)

Epoch 0, train: 0.004, dev: 0.000
Epoch 1, train: 0.223, dev: 0.591
Epoch 2, train: 0.761, dev: 0.639
Epoch 3, train: 0.792, dev: 0.647
Epoch 4, train: 0.805, dev: 0.654
Epoch 5, train: 0.821, dev: 0.663
Epoch 6, train: 0.831, dev: 0.663
Epoch 7, train: 0.842, dev: 0.661
Epoch 8, train: 0.850, dev: 0.662
Epoch 9, train: 0.863, dev: 0.669
Epoch 11, train: 0.884, dev: 0.666
Epoch 12, train: 0.888, dev: 0.671
Epoch 13, train: 0.891, dev: 0.673
Epoch 14, train: 0.893, dev: 0.668
Epoch 15, train: 0.893, dev: 0.672
Epoch 16, train: 0.895, dev: 0.674
Epoch 17, train: 0.896, dev: 0.672
Epoch 18, train: 0.894, dev: 0.667
Epoch 19, train: 0.893, dev: 0.668


In [10]:
lr001_lininterpa_dev_f1 = [0.000,0.591,0.639,0.647,0.654,0.663,0.663,0.661,0.662,0.669,0.666,0.671,0.673,0.668,0.672,0.674,0.672,0.667,0.668]

In [34]:
model2 = Train2BiLSTM(hidden_size=10)
model2.fit(train=(documents, doc_labels),
          train2=(pp_documents, pp_doc_labels),
          print_metrics=False, 
          learning_rate=0.003,
          dev = (dev_docs, dev_labels),
          epochs=20,
          alpha=0.5)

# saving model 2:
torch.save(model2, "nlp-project/models/Experiment2.pt")

Epoch 0, train: 0.000, dev: 0.000
Epoch 1, train: 0.304, dev: 0.605
Epoch 2, train: 0.790, dev: 0.640
Epoch 3, train: 0.817, dev: 0.645
Epoch 4, train: 0.831, dev: 0.648
Epoch 5, train: 0.840, dev: 0.652
Epoch 6, train: 0.845, dev: 0.654
Epoch 7, train: 0.848, dev: 0.658
Epoch 8, train: 0.855, dev: 0.665
Epoch 9, train: 0.867, dev: 0.670
Epoch 10, train: 0.878, dev: 0.670
Epoch 11, train: 0.883, dev: 0.674
Epoch 12, train: 0.886, dev: 0.675
Epoch 13, train: 0.889, dev: 0.671
Epoch 14, train: 0.891, dev: 0.673
Epoch 15, train: 0.894, dev: 0.675
Epoch 16, train: 0.896, dev: 0.675
Epoch 17, train: 0.897, dev: 0.678
Epoch 18, train: 0.898, dev: 0.673
Epoch 19, train: 0.900, dev: 0.678


In [14]:
lr003_a5_dev_f1 = [0.000,0.595,0.634,0.649,0.655,0.658,0.669,0.667,0.671,0.669,0.671,0.675,0.674,0.671,0.670,0.676,0.678,0.673,0.677,0.674]

In [29]:
model3 = Train2BiLSTM(hidden_size=10)
model3.fit(train=(documents, doc_labels),
          train2=(pp_documents, pp_doc_labels),
          print_metrics=False, 
          learning_rate=0.003,
          dev = (dev_docs, dev_labels),
          epochs=20,
          alpha=0.1)

Epoch 0, train: 0.000, dev: 0.000
Epoch 1, train: 0.012, dev: 0.112
Epoch 2, train: 0.703, dev: 0.642


KeyboardInterrupt: 

In [8]:
lr003_a1_dev_f1 = model3.dev_f1_log

In [25]:
dev_f1s = [lr001_lininterpa_dev_f1[:19], lr003_a5_dev_f1[:19], lr003_a1_dev_f1[:19]]

dev_f1s

[[0.0,
  0.591,
  0.639,
  0.647,
  0.654,
  0.663,
  0.663,
  0.661,
  0.662,
  0.669,
  0.666,
  0.671,
  0.673,
  0.668,
  0.672,
  0.674,
  0.672,
  0.667,
  0.668],
 [0.0,
  0.595,
  0.634,
  0.649,
  0.655,
  0.658,
  0.669,
  0.667,
  0.671,
  0.669,
  0.671,
  0.675,
  0.674,
  0.671,
  0.67,
  0.676,
  0.678,
  0.673,
  0.677],
 [0,
  0.6277541553923464,
  0.6399691952252599,
  0.6420038535645471,
  0.6438461538461538,
  0.6446407990779869,
  0.6563944530046225,
  0.655436447166922,
  0.6610234705656022,
  0.6648564778898371,
  0.66873786407767,
  0.6666666666666666,
  0.6661496704148896,
  0.6674427629025999,
  0.6723470178156468,
  0.6635730858468677,
  0.663839629915189,
  0.662848415425735,
  0.6610494063577173]]

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=b2f14aee-af04-4db5-af55-57a3a58b9f40' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>